# Config

In [3]:
import os

In [4]:
##### file system
INPUT_DIR = "/kaggle/input/audio-mnist/data"
WORKING_DIR = "/kaggle/working"

##### dataset
SAMPLE_RATE = 48_000
MAX_INPUT_SIZE = 65_536

##### training
TRAIN_PERCENTAGE = 0.8
BATCH_SIZE = 64
DATA_PERCENTAGE = 0.5

In [5]:
if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)

# Explore Data

In [6]:
from tqdm import tqdm
from scipy.io import wavfile
import shutil
import matplotlib.pyplot as plt
import pandas as pd

## Prepare Data for PyTorch

I will be using PyTorch, to load a custom dataset it needs some fixing.

- I will create a csv file with information regarding all files

In [7]:
def create_csv_info():
    # initialize the dataset
    dataset = pd.DataFrame(columns=["file", "label", "speaker"])

    for speaker in tqdm(os.listdir(INPUT_DIR)):
        # check if "speaker" is a folder
        # "speaker" should be a folder with all the recordings from the given
        #   speaker, the name of the speaker is the name of the folder
        if not os.path.isdir(os.path.join(INPUT_DIR, speaker)):
            continue

        for file in os.listdir(os.path.join(INPUT_DIR, speaker)):
            label = file.split("_")[0]
            file_path = os.path.join(speaker, file)

            dataset = dataset.append({
                "file": file_path,
                "label": label,
                "speaker": speaker
            }, ignore_index=True)

    # write dataset as csv
    dataset.to_csv(os.path.join(WORKING_DIR, "dataset.csv"), index=False)

In [8]:
create_csv_info()

100%|██████████| 61/61 [01:17<00:00,  1.27s/it]


# Load Dataset

In [9]:
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np


In [10]:
class AudioSample():
    # this defines one audio sample
    def __init__(self, sample_rate, samples):
        self.sample_rate = sample_rate
        self.samples = samples

    def display(self):
        plt.plot(self.samples)
        plt.show()


class AudioDescription():
    # apart from the audio also has the label and the speaker
    def __init__(self, audio_sample, label, speaker):
        self.audio_sample = audio_sample
        self.label = label
        self.speaker = speaker

    def display(self):
        self.audio_sample.display()
        print("Label: ", self.label)
        print("Speaker: ", self.speaker)


class AudioMNIST(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):
        csv_file_full_path = os.path.join(root_dir, csv_file)

        self.dataset = pd.read_csv(csv_file_full_path)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):

        if torch.is_tensor(idx):
            idx = idx.tolist()

        file_path = os.path.join(self.root_dir, self.dataset.iloc[idx, 0])

        sample_rate, samples = wavfile.read(file_path)

        audio_description = AudioDescription(
            AudioSample(sample_rate, samples),
            self.dataset.iloc[idx, 1],
            self.dataset.iloc[idx, 2]
        )

        if self.transform:
            audio_description = self.transform(audio_description)

        return audio_description


## Transforms

The data in dataset does not have all the same size. So transformations are
required. Two transformations are possible. Padding and Cropping.

In [11]:
class Padding():
    def __init__(self, output_size=MAX_INPUT_SIZE, mode="edge"):
        # output size is an integer
        # for mode check np.pad https://numpy.org/doc/stable/reference/generated/numpy.pad.html
        self.output_size = output_size
        self.mode = mode

    def __call__(self, audio_description):
        # sample is an AudioDescription
        if len(audio_description.audio_sample.samples) > self.output_size:
            # throw error; cannot pad sample to a smaller size
            raise ValueError("Cannot pad sample to a smaller size")

        new_samples = np.pad(audio_description.audio_sample.samples, ((self.output_size - len(audio_description.audio_sample.samples) + 1) //
                                                                      2, (self.output_size - len(audio_description.audio_sample.samples)) // 2), mode=self.mode)

        return AudioDescription(
            AudioSample(audio_description.audio_sample.sample_rate, new_samples),
            audio_description.label,
            audio_description.speaker
        )


class RandomCrop():
    def __init__(self, output_size=int(MAX_INPUT_SIZE/2)):
        # output size is an integer
        self.output_size = output_size

    def __call__(self, audio_description):
        # audio_sample is an AudioDescription
        if len(audio_description.audio_sample.samples) <= self.output_size:
            # pad the sample
            audio_description = Padding(self.output_size)(audio_description)
        else:
            # get random start index
            start_index = np.random.randint(
                0, len(audio_description.audio_sample.samples) - self.output_size)

            audio_description = AudioDescription(AudioSample(
                audio_description.audio_sample.sample_rate, audio_description.audio_sample.samples[start_index:start_index + self.output_size]),
                audio_description.label,
                audio_description.speaker
            )

        return audio_description


## Putting it all together

In [12]:
dataset = AudioMNIST(WORKING_DIR + "/dataset.csv",
                     INPUT_DIR, transform=RandomCrop())

# split dataset into train and test
train_size = int(TRAIN_PERCENTAGE * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, test_size])


def my_collate(batch):
    # batch is a list of AudioDescription
    samples = []
    labels = []
    speakers = []

    for audio_description in batch:
        samples.append(audio_description.audio_sample.samples)
        labels.append(audio_description.label)
        speakers.append(audio_description.speaker)

    # convert the list of audio samples to a tensor with shape [N, W]
    X = torch.tensor(samples)

    # add a new dimension to the tensor to get the desired shape [N, 1, W]
    X = X.unsqueeze(1)

    # convert the lists of labels and speakers to tensors
    y = torch.tensor(labels)
    z = torch.tensor(speakers)

    # return the modified tensors
    return X, y, z


dataloader_train = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, collate_fn=my_collate)
dataloader_test = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, collate_fn=my_collate)


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [13]:
# show one sample with the loader
for sample, label, speaker in dataloader_train:
    print(f"Shape of X [N, C, W]: {sample.shape}")
    print(f"Shape of y: {label.shape} {label.dtype}")
    break
    

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:230.)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:230.)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csr

Shape of X [N, C, W]: torch.Size([64, 1, 32768])
Shape of y: torch.Size([64]) torch.int64


# Model

In [14]:
from torch import nn

In [15]:
# get CPU or GPU device for training
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [41]:
class ConvClassifier(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv1d(
            in_channels=1, out_channels=32, kernel_size=9, stride=1, padding=1)
        self.pool1 = nn.MaxPool1d(2, stride=2)

        self.conv2 = nn.Conv1d(32, 64, 9, stride=1, padding=1)
        self.pool2 = nn.MaxPool1d(2, stride=2)

        self.conv3 = nn.Conv1d(64, 128, 9, stride=1, padding=1)
        self.pool3 = nn.MaxPool1d(2, stride=2)

        self.conv4 = nn.Conv1d(128, 256, 9, stride=1, padding=1)
        self.pool4 = nn.MaxPool1d(2, stride=2)

        self.conv5 = nn.Conv1d(256, 512, 9, stride=1, padding=1)
        self.pool5 = nn.MaxPool1d(2, stride=2)

        self.gap = nn.AdaptiveAvgPool1d(1)

        self.linear1 = nn.Linear(512, 256)
        self.linear2 = nn.Linear(256, 128)
        self.linear3 = nn.Linear(128, 10)

        self.relu = nn.ReLU()

    def forward(self, x):
        # pass to float
        x = x.float()

        x = self.pool1(self.relu(self.conv1(x)))
        x = self.pool2(self.relu(self.conv2(x)))
        x = self.pool3(self.relu(self.conv3(x)))
        x = self.pool4(self.relu(self.conv4(x)))
        x = self.pool5(self.relu(self.conv5(x)))

        x = self.gap(x)

        x = x.view(-1, 512)

        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.linear3(x)

        return x


In [42]:
model = ConvClassifier().to(device)
print(model)

ConvClassifier(
  (conv1): Conv1d(1, 32, kernel_size=(9,), stride=(1,), padding=(1,))
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(32, 64, kernel_size=(9,), stride=(1,), padding=(1,))
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(64, 128, kernel_size=(9,), stride=(1,), padding=(1,))
  (pool3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(128, 256, kernel_size=(9,), stride=(1,), padding=(1,))
  (pool4): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv1d(256, 512, kernel_size=(9,), stride=(1,), padding=(1,))
  (pool5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (gap): AdaptiveAvgPool1d(output_size=1)
  (linear1): Linear(in_features=512, out_features=256, bias=True)
  (linear2): Linear(in_features=256, out_features=128, bias=True)
  (linear3): Linear(i

# Training

In [43]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


In [44]:
def train(data_loader, model, loss_fn, optimizer):
    model.train()

    for batch, (sample, label, _) in enumerate(data_loader):
        sample, label = sample.to(device), label.to(device)

        # compute prediction error
        pred = model(sample)
        loss = loss_fn(pred, label)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(sample)
            print(f"loss: {loss:>7f}  [{current:>5d}/{len(data_loader.dataset):>5d}]")

def test(data_loader, model, loss_fn):
    model.eval()

    test_loss, correct = 0, 0

    with torch.no_grad():
        for sample, label, _ in data_loader:
            sample, label = sample.to(device), label.to(device)

            pred = model(sample)
            test_loss += loss_fn(pred, label).item()

            correct += (pred.argmax(1) == label).type(
                torch.float).sum().item()
            
    test_loss /= len(data_loader)
    correct /= len(data_loader.dataset)

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [46]:
epochs = 20

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}\n-------------------------------")
    
    train(dataloader_train, model, loss_fn, optimizer)
    test(dataloader_test, model, loss_fn)
    print()

print("Done!")

Epoch 1
-------------------------------


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:230.)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:230.)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csr

loss: 0.863717  [    0/24000]
loss: 1.023399  [ 6400/24000]
loss: 0.852181  [12800/24000]
loss: 0.833847  [19200/24000]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:230.)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:230.)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csr

Test Error: 
 Accuracy: 68.3%, Avg loss: 0.920539 


Epoch 2
-------------------------------


...

Test Error: 
 Accuracy: 86.8%, Avg loss: 0.383889 


Epoch 20
-------------------------------


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:230.)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:230.)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csr

loss: 0.521337  [    0/24000]
loss: 0.292680  [ 6400/24000]
loss: 0.350408  [12800/24000]
loss: 0.655213  [19200/24000]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:230.)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:230.)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csr

Test Error: 
 Accuracy: 90.3%, Avg loss: 0.322910 


Done!
